In [1]:
import datetime
import os
import pandas as pd
import json
import sklearn
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, f1_score, recall_score, precision_score
from sklearn.model_selection import train_test_split
from transformers import get_linear_schedule_with_warmup
import matplotlib.pyplot as plt

# function definitions to save logs to a csv file
def save_metrics_to_csv(json_filepath, csv_filename):
    with open(json_filepath) as file:
        data = json.load(file)

        log_history = data['log_history'] #focus on this column for history
        df = pd.DataFrame(log_history) # Convert the list of dictionaries to a DataFrame

        df.to_csv(csv_filename, index=False)


# Function to plot metrics
def plot_metrics_from_csv(csv_filepath, output_dir):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_filepath)

    plt.figure(figsize=(12, 8))

    # Plot accuracy
    plt.subplot(2, 3, 1)
    plt.plot(df['epoch'], df['eval_accuracy'], label='Accuracy')
    plt.title('Accuracy')
    plt.xlabel('epoch')
    plt.legend()

    # Plot precision
    plt.subplot(2, 3, 2)
    plt.plot(df['epoch'], df['eval_precision'], label='Precision')
    plt.title('Precision')
    plt.xlabel('epoch')
    plt.legend()

    # Plot recall
    plt.subplot(2, 3, 3)
    plt.plot(df['epoch'], df['eval_recall'], label='Recall')
    plt.title('Recall')
    plt.xlabel('epoch')
    plt.legend()

    # Plot F1 score
    plt.subplot(2, 3, 4)
    plt.plot(df['epoch'], df['eval_f1'], label='F1 Score')
    plt.title('F1 Score')
    plt.xlabel('epoch')
    plt.legend()

    # !!!! Plot training loss
    plt.subplot(2, 3, 5)
    plt.plot(df['epoch'], df['loss'], label='Training Loss')  # Make sure 'loss' column exists in your csv
    plt.title('Training Loss')
    plt.xlabel('epoch')
    plt.legend()

    # !!!! Plot validation loss
    plt.subplot(2, 3, 6)
    plt.plot(df['epoch'], df['eval_loss'], label='Validation Loss')  # Make sure 'eval_loss' column exists in your csv
    plt.title('Validation Loss')
    plt.xlabel('epoch')
    plt.legend()


    plt.tight_layout()

    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    plt.savefig(os.path.join(output_dir, 'metrics_plot.png'))
    plt.show()


def compute_metrics(pred): #evaluation metrics function
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    # calculate precision, recall, f1 score, and accuracy
    precision = precision_score(labels, preds, average='weighted')
    recall = recall_score(labels, preds, average='weighted')
    f1 = f1_score(labels, preds, average='weighted')
    accuracy = accuracy_score(labels, preds)
    return {
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

def get_latest_checkpoint(folder_path):
    # Get a list of all files and directories in the specified folder
    files_and_dirs = os.listdir(folder_path)

    # Filter only directories (assumed to be checkpoints)
    checkpoint_dirs = [d for d in files_and_dirs if os.path.isdir(os.path.join(folder_path, d))]

    if not checkpoint_dirs:
        print("No checkpoint directories found.")
        return None

    # Extract the checkpoint numbers from the directory names
    checkpoint_numbers = [int(d.split('-')[1]) for d in checkpoint_dirs]

    # Identify the directory with the highest checkpoint number
    latest_checkpoint = os.path.join(folder_path, f"checkpoint-{max(checkpoint_numbers)}")

    return latest_checkpoint

In [2]:
from sklearn.model_selection import train_test_split
import pandas as pd
import os

dataset = pd.read_csv("/content/PREPROCESSED-NOTES.csv")

text_data = dataset["text"].to_list()

sdoh_data = {
    "sdoh_community_present": dataset["sdoh_community_present"].to_list(),
    "sdoh_community_absent": dataset["sdoh_community_absent"].to_list(),
    "sdoh_education": dataset["sdoh_education"].to_list(),
    "sdoh_economics": dataset["sdoh_economics"].to_list(),
    "sdoh_environment": dataset["sdoh_environment"].to_list(),
    "behavior_alcohol": dataset["behavior_alcohol"].to_list(),
    "behavior_tobacco": dataset["behavior_tobacco"].to_list(),
    "behavior_drug": dataset["behavior_drug"].to_list()
}

base_path = 'test_train_split/behavior_drug'
os.makedirs(base_path, exist_ok=True)

# Iterate through each SDOH data category
for category, data in sdoh_data.items():
    # Create folder for each category
    base_path = f"test_train_split/{category}"
    os.makedirs(base_path, exist_ok=True)

    # Split data for the current category
    X_train, X_val, y_train, y_val = train_test_split(
        text_data, data, random_state=0, train_size=0.8, stratify=data
    )

        # Save all splits as CSV files
    pd.DataFrame({"text": X_train}).to_csv(f"{base_path}/X_train.csv", index=False)
    pd.DataFrame({"text": X_val}).to_csv(f"{base_path}/X_val.csv", index=False)
    pd.DataFrame({category: y_train}).to_csv(f"{base_path}/y_train.csv", index=False)
    pd.DataFrame({category: y_val}).to_csv(f"{base_path}/y_val.csv", index=False)

In [4]:
!pip install plotting transformers[torch] accelerate -U
import datetime
import os
import torch
from torch.optim import AdamW  # variant of Adam with weight decay
from torch.utils.data import Dataset
from transformers import GPT2ForSequenceClassification, GPT2Tokenizer, TrainingArguments, Trainer
import pandas as pd
import json
import sklearn
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, f1_score, recall_score, precision_score
from sklearn.model_selection import train_test_split
from transformers import get_linear_schedule_with_warmup

from plotting import *

# Initialize tokenizer, this is standard approach with GPT-2
# Loading GPT-2 tokenizer and model for sequence classification
tokenizer = GPT2Tokenizer.from_pretrained("gpt2", num_labels = 5)
# GPT-2 uses the same token for end-of-sentence and padding.
tokenizer.pad_token = tokenizer.eos_token  # Set pad token

# Ensure the model is compatible with the tokenizer settings
configuration = GPT2ForSequenceClassification.config_class.from_pretrained("gpt2", num_labels= 5)
configuration.pad_token_id = tokenizer.pad_token_id
model = GPT2ForSequenceClassification(configuration)
# model = GPT2ForSequenceClassification.from_pretrained("gpt2")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


# Set up no decay for certain model parameters to avoid regularization on them
no_decay = ['bias', 'LayerNorm.weight']  # weight decay with a minor penalty during
optimizer_grouped_parameters = [  # no selects params added
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=1e-5)
dataset = pd.read_csv("/content/PREPROCESSED-NOTES.csv")

text_data = dataset["text"].to_list()
sdoh_data = dataset["behavior_alcohol"].to_list()

timestamp_fortrain = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(text_data, sdoh_data, random_state=0, train_size=0.8,
                                                  stratify=sdoh_data)
max_seq_length = 100  # actually 50 but increase to accomadate outliers

# Calculate the number of trainable parameters in the model
num_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
model_size_MB = num_trainable_params * 4 / (1024 ** 2)
effective_batch = 8 / (50*4*model_size_MB) #gpu/seqlength * 4 * model size

train_encodings = tokenizer(X_train, truncation=True, padding='max_length', max_length=max_seq_length, return_tensors='pt')
val_encodings = tokenizer(X_val, truncation=True, padding='max_length', max_length=max_seq_length, return_tensors='pt')

# custom Dataset class for loading training and validation data
class DataLoader(Dataset):
    def __init__(self, encodings, labels):
        """ :rtype: object """
        self.encodings = encodings
        self.labels = torch.tensor(labels, dtype=torch.long)  # Converting to tensor , maybe use just 'labels'

    def __getitem__(self, idx):
        try:
            item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
            item['labels'] = self.labels[idx].clone().detach()  # Already a tensor, just clone and detach
            return item
        except Exception as e:
            print(f"index error: {idx}: {e}")
            return None

    def __len__(self):
        return len(self.labels) # detach from tensor device

# Initialize the DataLoader for training and validation sets with the tokenized encodings
train_dataset: DataLoader = DataLoader(
    train_encodings,  # These should be the output from the tokenizer
    y_train  # These should be your labels, as a list or tensor
)

val_dataset = DataLoader(
    val_encodings,  # These should be the output from the tokenizer
    y_val  # These should be your labels, as a list or tensor
)

tensor_logs = f'./logs/tensor_logs/{timestamp_fortrain}' #create seperate logs for tensor/epoch
os.makedirs(tensor_logs, exist_ok=True)
epoch_logs = f'./logs/epoch_logs/{timestamp_fortrain}'
os.makedirs(epoch_logs, exist_ok=True)

# training args - need to adjust
training_args = TrainingArguments(
    output_dir= epoch_logs,  # change to epoch log directory, convert to a text
    logging_strategy='epoch',  # characterize as epoch
    num_train_epochs=7,
    #per_device_train_batch_size=64,  # cpu constraint,  64 approp
    per_device_train_batch_size=16, #reduced batch sie
    per_device_eval_batch_size=64,  # gradient accum if batch size of two, 64 approp
    save_strategy= 'epoch',
    warmup_steps=500,
    weight_decay=1e-5,
    logging_dir= tensor_logs,  # change to tensor logs
    eval_steps=100,
    evaluation_strategy="epoch",
    #accumulate gradients over 4 steps
    gradient_accumulation_steps = 4

)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

trainer.train()
evaluation_results = trainer.evaluate()

#readable results
latest_checkpoint = get_latest_checkpoint(epoch_logs) # latest checkpoint update to csv
json_path = os.path.join(latest_checkpoint, 'trainer_state.json')
save_metrics_to_csv(json_path, 'eval_metric.csv') #update metrics
plot_metrics_from_csv('eval_metric.csv', 'graphs')

save_directory = "saved_models/gpt2"

os.makedirs(save_directory, exist_ok=True)
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

print("Evaluation Results:", evaluation_results)

ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pyparsing/core.py", line 817, in _parseNoCache
    loc, tokens = self.parseImpl(instring, pre_loc, doActions)
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pyparsing/core.py", line 4025, in parseImpl
    raise maxException
pip._vendor.pyparsing.exceptions.ParseException: Expected {Re:("\n        (?P<operator>(~=|==|!=|<=|>=|<|>|===))\n        (?P<version>\n            (?:\n                # The identity operators allow for an escape hatch that will\n                # do an exact string match of the version you wish to install.\n                # This will not be parsed by PEP 440 and we cannot determine\n                # any semantic meaning from it. This operator is discouraged\n                # but included entirely as an escape hatch.\n                (?<====)  # Only match for the identity operator\n                \s*\n                [^

KeyboardInterrupt: 